In [0]:
# new logs
# finetune start = 159 epoch
# use_mean beta at finetune

In [0]:
!pip install faiss-gpu==1.6.1
!pip install tensorboardX
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
!pip install mlflow

In [0]:
import torch
from tensorboardX import SummaryWriter
import numpy as np
import torch.nn as nn
from torch.nn import init
# from apex import amp
import torch.nn.functional as f
import torchvision
from math import ceil
from PIL import Image
import os
from torchvision import transforms
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_
from scipy.optimize import linear_sum_assignment as lsa
import seaborn as sns
import time
import matplotlib.pyplot as plt
import json
from datetime import datetime
import tarfile
import scipy.io
import faiss
import warnings
from warmup_scheduler import GradualWarmupScheduler
import gc
from scipy.special import softmax
import mlflow
from collections import Counter
from shutil import copyfile
warnings.filterwarnings('ignore')
torch.cuda.empty_cache()

# params.py

sop_dataset_path = '/kaggle/input/sopdataset/Stanford_Online_Products/'
inshop_dataset_path = '/kaggle/input/inshopclothes/In_shop Clothes Retrieval Benchmark'
cub_dataset_path = '/kaggle/input/cub200/CUB_200_2011'
cars_dataset_path = '/kaggle/input/cars196'
pretrained_pytorch_models = '/kaggle/input/pretrained-pytorch-models'
input_dir = '/kaggle/input/checkpoints'

checkpoints_dir = '/kaggle/working/'
tensorboard_dir = '/kaggle/working/'
losses_dir = '/kaggle/working/'
metrics_dir = '/kaggle/working/'
density_plots_dir = '/kaggle/working/'

device = 'cuda:0'
random_seed = 0
lr = 5e-5
embedding_dim = 128
batch_size = 80
masks_lr = 5e-5

# OnlineProducts params
sop_train_classes = range(0, 11318)
sop_test_classes = range(11318, 22634)
sop_nb_train_data = 59551

# InShop params
inshop_train_classes = range(0, 3997)
inshop_test_classes = range(0, 3985)
inshop_nb_train_data = 25882

# CARS196 params
cars_train_classes = range(1, 99)
cars_test_classes = range(99, 197)
cars_nb_train_data = 8054

# CUB200 params
cub_train_classes = range(1, 101)
cub_test_classes = range(101, 201)
cub_nb_train_data = 5864

epochs = 250
fine_tune_epoch = 250 # 190
recluster_epoch_freq = 2

load_saved_model_for_test = True
save_clusters = True
save_model = True
to_make_density_plots = True

continue_training_from_epoch = True
checkpoint_from_epoch = 211
clusters_from_epoch = 210
load_all_metrics = True  # True if continue training
all_metrics_dir = f'/kaggle/input/checkpoints/all_metrics_{checkpoint_from_epoch}'

# to eval model with loaded checkpoint or to check model performance before training
compute_metrics_before_training = False
evalaute_on_train_data = False
lambda1 = 5e-4  # masks l1 regularization
lambda2 = 5e-3

available_datasets = [
    'OnlineProducts',
    'InShopClothes',
    'CUB200',
    'CARS196'
]
cur_dataset = 'OnlineProducts'
use_gem = True
use_lr_scheduler = True
use_mean_beta_at_finetune = True  # False
use_cls_loss = False

if cur_dataset is 'CUB200' or cur_dataset is 'CARS196':
    nb_clusters = 4
else:
    nb_clusters = 8
    
# ft_epoch = 60 if cur_dataset is 'CUB200' or cur_dataset is 'CARS196' else 80
ft_epoch = 159  # 100

# MLFlow params
working_dir = '/kaggle/working/'
params_save = {key: value for key, value in locals().items() if 'gc' not in key and not key.startswith('__')}
mlflow.set_experiment('SOP masks baseline relu')
for key, value in params_save.items():
    mlflow.log_param(key, value)

    # artifact_path = mlflow.get_artifact_uri()
# mlflow.log_artifact(os.path.abspath(__file__))
dirs_for_logs = ['distances', 'masks', 'loss', 'clusters', 'eval', 'artifacts']
for dir_ in dirs_for_logs:
    if dir_ not in os.listdir(metrics_dir):
        os.makedirs(os.path.join(metrics_dir, dir_))

# loading all previous metrics logs in Kaggle
if load_all_metrics:
    dirs_to_make = os.listdir(all_metrics_dir)
    for dir_ in dirs_to_make:
        if dir_ not in os.listdir(metrics_dir):
            os.makedirs(os.path.join(metrics_dir, dir_))
    for dir_ in dirs_to_make:
        metrics_to_load = os.listdir(os.path.join(all_metrics_dir, dir_))
        for metr in metrics_to_load:
            copyfile(os.path.join(all_metrics_dir, os.path.join(dir_, metr)), os.path.join(metrics_dir, os.path.join(dir_, metr)))
    print(f'loaded previous metrics logs')

def log_metrics(path, metrics, step, with_mlflow=True):
    time_step = str(time.time()).replace('.', '')[:13]
    with open(os.path.join(metrics_dir, path), 'a') as f:
        f.write(f'{time_step} {metrics} {step}\n')
    if with_mlflow:
        mlflow.log_metric(path, metrics, step)

def log_artifacts(artifact_path, file_name, with_mlflow=True):
    copyfile(artifact_path, os.path.join(os.path.join(metrics_dir, 'artifacts'), file_name))
    if with_mlflow:
        mlflow.log_artifact(artifact_path)    


from math import ceil
import numpy as np
import torchvision
import torch
from torch.nn import Linear, Dropout, AvgPool2d, MaxPool2d
from torch.nn.init import xavier_normal_

def gem(x, p=3, eps=1e-6):
    return torch.flatten(f.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1. / p), 1)
    # return F.lp_pool2d(F.threshold(x, eps, eps), p, (x.size(-2), x.size(-1))) # alternative

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = torch.nn.parameter.Parameter(torch.ones(1) * p, requires_grad=True)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)

    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(
            self.eps) + ')'

def get_resnet50():
    resnet50 = torchvision.models.resnet50(pretrained=True)
    resnet50.features = torch.nn.Sequential(resnet50.conv1, resnet50.bn1, resnet50.relu, resnet50.maxpool, resnet50.layer1,
                                            resnet50.layer2, resnet50.layer3, resnet50.layer4)
    resnet50.sz_features_output = 2048
    for module in filter(lambda m: type(m) == torch.nn.BatchNorm2d, resnet50.modules()):
        module.eval()
        module.train = lambda _: None
    return resnet50


def get_params_dict(model, emb_module_name=['embedding', 'masks']):
    dict_ = {k: [] for k in ['backbone', *emb_module_name]}
    for name, param in model.named_parameters():
        name = name.split('.')[0]
        if name not in emb_module_name:
            dict_['backbone'] += [param]
        else:
            dict_[name] += [param]
    nb_total = len(list(model.parameters()))
    nb_dict_params = sum([len(dict_[d]) for d in dict_])
    assert nb_total == nb_dict_params
    return dict_


def get_embedding(model):
    if use_gem:
        model.features_pooling = GeM().to(device)
    else:
        model.features_pooling = AvgPool2d(7, stride=1, padding=0, ceil_mode=True, count_include_pad=True)
    model.features_dropout = Dropout(0.01)
    torch.random.manual_seed(1)
    model.embedding = Linear(model.sz_features_output, embedding_dim).to(list(model.parameters())[0].device)
    model.classification_layer = nn.Linear(model.sz_features_output, len(sop_train_classes)).to(device)
    model.masks = torch.nn.Embedding(nb_clusters, embedding_dim)
    # initialize weights
    model.masks.weight.data.normal_(0.9, 0.7)  # 0.1, 0.005
    torch.random.manual_seed(1)
    np.random.seed(1)
    # _xavier_init
    model.embedding.weight.data = xavier_normal_(model.embedding.weight.data, gain=1)
    features_parameters = model.features.parameters()
    model.parameters_dict = get_params_dict(model=model)

    def forward(x, with_embedding_layer=True, learner_id=None, with_softmax=False):
        x = model.features(x)
        x = model.features_pooling(x)
        x = model.features_dropout(x)
        features = x.view(x.size(0), -1)
        if with_embedding_layer:
            x = model.embedding(features)
            if with_softmax:
                return x, model.classification_layer(features)
            return x
        else:
            embeddings = model.embedding(features)
            return features, embeddings
    model.forward = forward


def get_model():
    resnet50 = get_resnet50()
    get_embedding(resnet50)
    return resnet50

#  OnlineProducts Dataset

class OnlineProductsDataset(torch.utils.data.Dataset):
    def __init__(self, ds_path, mode, transforms_mode='test', train_st_class=0, train_fin_class=11318, test_st_class=11318, test_fin_class=22634,
                 nb_train_imgs=59551, nb_test_imgs=60502):
        self.mode = mode
        self.transforms_mode = transforms_mode
        self.sz_crop = 224
        self.sz_resize = 256
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        if self.mode == 'train':
            self.classes = range(train_st_class, train_fin_class)
        else:
            self.classes = range(test_st_class, test_fin_class)
        self.ds_path = ds_path
        self.labels, self.img_paths, self.idxs = list(), list(), list()

        im_paths_list = os.path.join(ds_path, f'Ebay_{self.mode}.txt')
        with open(im_paths_list) as f:
            f.readline()
            idx, nb_imgs = 0, 0

            for (image_id, class_id, _, path) in map(str.split, f):
                nb_imgs += 1
                if int(class_id) - 1 in self.classes:
                    self.img_paths.append(os.path.join(self.ds_path, path))
                    self.labels.append(int(class_id) - 1)
                    self.idxs.append(idx)
                    idx += 1

        if self.mode == 'train':
            assert nb_imgs == nb_train_imgs
        else:
            assert nb_imgs == nb_test_imgs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        im = Image.open(self.img_paths[index])
        if len(list(im.split())) == 1:
            im = im.convert('RGB')
        im = self.apply_augmentation(im)
        return im, self.labels[index], index

    def apply_augmentation(self, im):
        if self.transforms_mode == 'train':
            transforms_ = transforms.Compose([
                transforms.RandomResizedCrop(self.sz_crop),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        else:
            transforms_ = transforms.Compose([
                transforms.Resize(self.sz_resize),
                transforms.CenterCrop(self.sz_crop),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        return transforms_(im)

    def get_nb_classes(self):
        return len(self.classes)

    def get_label(self, index):
        return self.labels[index]

    def get_within_indexes(self, indexes):
        self.labels = [self.labels[i] for i in indexes]
        self.idxs = [self.idxs[i] for i in indexes]   ###
        self.img_paths = [self.img_paths[i] for i in indexes]


# InShopClothes Dataset

class InShopDataset(torch.utils.data.Dataset):
    def __init__(self, ds_path, transforms_mode='test', ds_info_path='Eval/list_eval_partition.txt', mode='train', nb_train_images=25882,
                 nb_query_images=14218, nb_gallery_images=12612, nb_train_classes=3997, nb_query_classes=3985,
                 nb_gallery_classes=3985):

        ''' nb_all_images = 52712 '''
        self.ds_info_path = os.path.join(ds_path, ds_info_path)
        self.mode = mode
        self.transforms_mode = transforms_mode
        self.paths, self.labels, self.ids = [], [], []

        self.sz_crop = 224
        self.sz_resize = 256
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        with open(self.ds_info_path, 'r') as f:
            ds_info = f.readlines()[2:]

        for i, inf_ in enumerate(ds_info):
            inf_ = inf_.replace('\n', '').split()
            path, lbl, mode = inf_
            for el in [path, lbl, mode]:
                assert el != ''
            if mode == self.mode:
                self.paths.append(os.path.join(ds_path, path))
                self.labels.append(int(lbl[3:]))
                self.ids.append(i)

        ordered_labels = {lbl: i for i, lbl in enumerate(sorted(set(self.labels)))}
        self.labels = list(map(lambda l: ordered_labels[l], self.labels))
        if self.mode is 'train':
            nb_images, nb_classes = nb_train_images, nb_train_classes
        elif self.mode is 'query':
            nb_images, nb_classes = nb_query_images, nb_query_classes
        else:
            nb_images, nb_classes = nb_gallery_images, nb_gallery_classes
        assert len(self.labels) == nb_images
        assert len(set(self.labels)) == nb_classes

    def apply_augmentation(self, img):
        if self.transforms_mode == 'train':
            transforms_ = transforms.Compose([
                transforms.RandomResizedCrop(self.sz_crop),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        else:
            transforms_ = transforms.Compose([
                transforms.Resize(self.sz_resize),
                transforms.CenterCrop(self.sz_crop),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        return transforms_(img)

    def __len__(self):
        return len(self.labels)

    def get_nb_classes(self):
        return len(set(self.labels))

    def unique_labels(self):
        return set(self.labels)

    def __getitem__(self, idx):
        self.paths[idx] = self.paths[idx].replace('/img/', '/Img/')
        assert os.path.exists(self.paths[idx])
        try:
            # assert os.path.exists(self.paths[idx])
            img = Image.open(self.paths[idx])
        except:
            print(self.paths[idx])

        if len(list(img.split())) != 3:
            img = img.convert('RGB')
        img = self.apply_augmentation(img)
        return img, self.labels[idx], idx

    def get_label(self, idx):
        return self.labels[idx]

    def get_within_indexes(self, idxs):
        self.paths = [self.paths[i] for i in idxs]
        self.labels = [self.labels[i] for i in idxs]
        self.ids = [self.ids[i] for i in idxs]

# CUB200 Dataset

class CUBDataset(torch.utils.data.Dataset):
    def __init__(self, ds_path, mode, transforms_mode='test', nb_train_images=5864, nb_test_images=5924):

        self.ds_path = ds_path
        self.mode = mode
        self.transforms_mode = transforms_mode
        self.paths, self.labels, self.ids = [], [], []

        self.sz_crop = 224
        self.sz_resize = 256
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        with open(os.path.join(self.ds_path, 'images.txt'), 'r') as f:
            images = f.readlines()

        with open(os.path.join(self.ds_path, 'image_class_labels.txt'), 'r') as f:
            image_class_labels = f.readlines()

        for labels, paths in zip(image_class_labels, images):
            id1, label = labels.replace('\n', '').split()
            id2, path = paths.replace('\n', '').split()
            assert int(id1) == int(id2)

            # using first 100 classes for training
            if mode is 'train' and int(label) < 101:
                self.labels.append(int(label))
                self.paths.append(path)
                self.ids.append(int(id1) - 1)

            # and second 100 classes for testing
            elif mode is 'test' and int(label) >= 101:
                self.labels.append(int(label))
                self.paths.append(path)
                self.ids.append(int(id1) - 1)

        if mode is 'train':
            assert len(self.labels) == nb_train_images
        else:
            assert len(self.labels) == nb_test_images

    def apply_augmentation(self, img):
        if self.transforms_mode == 'train':
            transforms_ = transforms.Compose([
                transforms.RandomResizedCrop(self.sz_crop),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        else:
            transforms_ = transforms.Compose([
                transforms.Resize(self.sz_resize),
                transforms.CenterCrop(self.sz_crop),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        return transforms_(img)

    def __len__(self):
        return len(self.labels)

    def get_nb_classes(self):
        return len(set(self.labels))

    def unique_labels(self):
        return set(self.labels)

    def __getitem__(self, idx):
        img = Image.open(self.ds_path + '/images/' + self.paths[idx])
        if len(list(img.split())) != 3:
            img = img.convert('RGB')
        img = self.apply_augmentation(img)
        return img, self.labels[idx], idx

    def get_label(self, idx):
        return self.labels[idx]

    def get_within_indexes(self, idxs):
        self.paths = [self.paths[i] for i in idxs]
        self.labels = [self.labels[i] for i in idxs]
        self.ids = [self.ids[i] for i in idxs]

# CARS196 Dataset

class CARSDataset(torch.utils.data.Dataset):
    def __init__(self, ds_path, mode, transforms_mode='test', nb_train_images=8054, nb_test_images=8131, nb_all=16185):

        self.ds_path = ds_path
        self.mode = mode
        self.transforms_mode = transforms_mode
        self.paths, self.labels, self.ids = [], [], []

        self.sz_crop = 224
        self.sz_resize = 256
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        mat = scipy.io.loadmat(os.path.join(ds_path, 'cars_annos.mat'))
        id = 0
        for annotation in mat['annotations'][0]:
            label = int(str(np.squeeze(annotation['class'])))
            if mode is 'train' and label < 99:
                self.paths.append(str(np.squeeze(annotation['relative_im_path'])))
                self.labels.append(int(str(np.squeeze(annotation['class']))))
                self.ids.append(id)
                id += 1
            elif mode is 'test' and label >= 99:
                self.paths.append(str(np.squeeze(annotation['relative_im_path'])))
                self.labels.append(int(str(np.squeeze(annotation['class']))))
                self.ids.append(id)
                id += 1

        if mode is 'train':
            assert len(self.labels) == nb_train_images
        else:
            assert len(self.labels) == nb_test_images

    def apply_augmentation(self, img):
        if self.transforms_mode == 'train':
            transforms_ = transforms.Compose([
                transforms.RandomResizedCrop(self.sz_crop),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        else:
            transforms_ = transforms.Compose([
                transforms.Resize(self.sz_resize),
                transforms.CenterCrop(self.sz_crop),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=self.mean,
                    std=self.std,
                )
            ])
        return transforms_(img)

    def __len__(self):
        return len(self.labels)

    def get_nb_classes(self):
        return len(set(self.labels))

    def unique_labels(self):
        return set(self.labels)

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.ds_path, self.paths[idx]))
        if len(list(img.split())) != 3:
            img = img.convert('RGB')
        img = self.apply_augmentation(img)
        return img, self.labels[idx], idx

    def get_label(self, idx):
        return self.labels[idx]

    def get_within_indexes(self, idxs):
        self.paths = [self.paths[i] for i in idxs]
        self.labels = [self.labels[i] for i in idxs]
        self.ids = [self.ids[i] for i in idxs]

class BatchSampler(torch.utils.data.sampler.BatchSampler):
    def __init__(self, dataset, batch_size=batch_size, m=4):
        self.m = m
        self.dataset = dataset
        self.batch_size = batch_size

        self.prod_labels = np.array(dataset.labels)
        self.unique_prod_labels = list(set(self.prod_labels))

        self.prod_label_indices_within_cur_label = self.get_all_prod_labels_indices_with_cur_label()
        self.nb_classes_to_pick = batch_size // m

        self.indices_real_from_class = self.get_all_prod_labels_indices_with_cur_label()
        self.indices_taken_from_class = np.zeros(len(self.unique_prod_labels))

    def get_all_prod_labels_indices_with_cur_label(self):
        out = []
        for c in self.unique_prod_labels:
            prod_label_indices = np.where(self.prod_labels == c)[0]
            np.random.shuffle(prod_label_indices)
            out.append(prod_label_indices)
        return out

    def __len__(self):
        return len(self.dataset) // self.batch_size

    def __iter__(self):
        for _ in range(len(self.dataset) // self.batch_size):
            if len(self.unique_prod_labels) > self.nb_classes_to_pick:
                replace = False
            else:
                replace = True
            chosen_classes = np.random.choice(
                len(self.unique_prod_labels), self.nb_classes_to_pick,
                replace=replace)
            out = []
            for cl in chosen_classes:
                if self.indices_taken_from_class[cl] + self.m \
                        < len(self.indices_real_from_class[cl]):
                    st = int(self.indices_taken_from_class[cl])
                    chosen_indices = self.indices_real_from_class[cl][st:st + self.m]
                else:
                    chosen_indices = np.random.choice(self.indices_real_from_class[cl],
                                                      self.m,
                                                      replace=len(self.indices_real_from_class[cl]) < self.m)
                out.extend(chosen_indices)
                self.indices_taken_from_class[cl] += self.m

                if self.indices_taken_from_class[cl] + self.m \
                        > len(self.indices_real_from_class[cl]):
                    np.random.shuffle(self.indices_real_from_class[cl])
                    self.indices_taken_from_class[cl] = 0
            yield out


# loss.py

class MarginLoss(torch.nn.Module):
    def __init__(self, beta=1.2, margin=0.2, nu=0.0, dist_weighted_sampler_thr=0.5):
        super(MarginLoss, self).__init__()
        self.beta = torch.nn.Parameter(torch.tensor([beta]), requires_grad=True)
        self.margin = margin
        self.nu = nu
        self.thr = dist_weighted_sampler_thr

    @staticmethod
    def get_distance(x):
        mm = torch.mm(x, x.t())
        dist = mm.diag().view((mm.diag().size()[0], 1))
        dist = dist.expand_as(mm)
        dist_ = dist + dist.t()
        dist_ = (dist_ - 2 * mm).clamp(min=0)
        return dist_.clamp(min=1e-4).sqrt()

    def sample_triplets(self, embeddings, prod_labels, iter_=None, learner_id=None, for_cluster=False):
        anchor_ids, pos_ids, neg_ids = [], [], []
        if not torch.is_tensor(prod_labels):
            prod_labels = torch.tensor(prod_labels)

        distance = self.get_distance(embeddings)
        p0 = prod_labels.clone().view(1, prod_labels.size()[0]).expand_as(distance)
        p1 = prod_labels.view(prod_labels.size()[0], 1).expand_as(distance)
        positives_ids = torch.eq(p0, p1).to(device, dtype=torch.float32) - (torch.eye(len(distance))).to(device)
        
        n_ids = ((positives_ids > 0) + (distance < self.thr)).to(device, dtype=torch.float32)
        unused_neg_part = 0
        if not for_cluster:
            n_ids_all = (distance[positives_ids == 0]).to(device, dtype=torch.float32)
            unused_neg_part = (torch.sum(n_ids)/torch.sum(n_ids_all)).item()
            log_metrics('clusters/unused_neg_part_within_cutoff', unused_neg_part, iter_)
            log_metrics(f'clusters/unused_neg_part_within_cutoff_learner_id_{learner_id}', unused_neg_part, iter_)   
                    
        total_neg_dist = torch.mean(distance[positives_ids == 0])
        negatives_ids = n_ids * 1e6 + distance
        to_retrieve_ids = max(1, min(int(positives_ids.data.sum()) // len(positives_ids), negatives_ids.size(1)))
        negatives = negatives_ids.topk(to_retrieve_ids, dim=1, largest=False)[1]
        negatives_ids_ = torch.zeros_like(negatives_ids.data).scatter(1, negatives, 1.0)

        for i in range(len(distance)):
            anchor_ids.extend([i] * (int(positives_ids.data.sum()) // len(positives_ids)))
            pos_ids_ = np.atleast_1d(positives_ids[i].nonzero().squeeze().cpu().numpy())
            neg_ids_ = np.atleast_1d(negatives_ids_[i].nonzero().squeeze().cpu().numpy())
            pos_ids.extend(pos_ids_)
            neg_ids.extend(neg_ids_)

            if len(anchor_ids) != len(pos_ids) or len(anchor_ids) != len(neg_ids):
                t = min(map(len, [anchor_ids, pos_ids, neg_ids]))
                anchor_ids = anchor_ids[:t]
                pos_ids = pos_ids[:t]
                neg_ids = neg_ids[:t]
        anchors, positives, negatives = embeddings[anchor_ids], embeddings[pos_ids], embeddings[neg_ids]
        return anchor_ids, anchors, positives, negatives, total_neg_dist, unused_neg_part

    def forward(self, embeddings, product_labels, iter_, learner_id=-1, for_cluster=False):
        a_indices, anchors, positives, negatives, total_neg_dist, unused_neg_part = self.sample_triplets(embeddings, product_labels, iter_=iter_, learner_id=learner_id, for_cluster=for_cluster)
        log_metrics('loss/beta', self.beta.data.tolist()[0], iter_)
        log_metrics(f'loss/beta_{learner_id}', self.beta.data.tolist()[0], iter_)
        
        beta_reg_loss = torch.norm(self.beta, p=1) * self.nu if a_indices is not None else 0.0
        d_ap = torch.sqrt(torch.sum((positives - anchors) ** 2, dim=1) + 1e-8)
        d_an = torch.sqrt(torch.sum((negatives - anchors) ** 2, dim=1) + 1e-8)
        d_pn = torch.sqrt(torch.sum((positives - negatives) ** 2, dim=1) + 1e-8)
        pos_loss = torch.clamp(d_ap - self.beta[0] + self.margin, min=0.0)
        neg_loss = torch.clamp(self.beta[0] - d_an + self.margin, min=0.0)
        loss = (torch.sum(pos_loss + neg_loss) + beta_reg_loss) / int(torch.sum((pos_loss > 0.0) + (neg_loss > 0.0)))
        log_metrics('loss/margin_loss', loss.item(), iter_)
        if not for_cluster:
            return loss, d_ap, d_an, d_pn, torch.sum(pos_loss > 0.0)/float(pos_loss.view(-1).size(0)), torch.sum(neg_loss > 0.0)/float(neg_loss.view(-1).size(0)), \
               float(neg_loss.view(-1).size(0)), total_neg_dist, int(torch.sum((pos_loss > 0.0) + (neg_loss > 0.0))), self.beta.data.tolist()[0], unused_neg_part
        else:
            return loss


def make_density_plots(epoch, pos_distances, neg_distances, path_to_save=density_plots_dir, save_fig=True,
                       show=False, mode='valid', log_distances=False):
    sns.set(color_codes=True)
    sns.kdeplot(pos_distances, shade=True, color='r', label='anchor-positive')
    sns.kdeplot(neg_distances, shade=True, color='b', label='anchor-negative')
    plt.xlabel('Distance')
    plt.ylabel('Density')
    if save_fig:
        plt.savefig(os.path.join(path_to_save, mode + f'_density_plot_at_epoch_{epoch}'))
        if log_distances:
            json.dump(list(pos_distances),
                      open(os.path.join(path_to_save, 'a_p_dists_' + mode + f'_epoch_{epoch}'), 'w'))
            json.dump(list(neg_distances),
                      open(os.path.join(path_to_save, 'a_n_dists_' + mode + f'_epoch_{epoch}'), 'w'))
        log_artifacts(os.path.join(path_to_save, mode + f'_density_plot_at_epoch_{epoch}.png'), mode + f'_density_plot_at_epoch_{epoch}.png')
    if show:
        plt.show()
    plt.clf()


def get_criterion():
    _criterion = [MarginLoss().cuda() for _ in range(nb_clusters+1)]
    return _criterion


def get_optimizer(_model_, beta):
    opt_ = torch.optim.Adam([
        {'params': _model_.parameters_dict['backbone'], 'lr': lr},  # 'weight_decay': weight_decay},
        {'params': _model_.parameters_dict['embedding'], 'lr': lr},  # 'weight_decay': weight_decay},
        {'params':  _model_.parameters_dict['masks'], 'lr': masks_lr},
        {'params': beta, 'lr': 0.01, 'weight_decay': 0.0001}])
    return opt_


def kmeans(x, num_clusters, previous_centroids=None, nrns_clustering=False):
    x = np.asarray(x.reshape(x.shape[0], -1), dtype=np.float32)
    if nrns_clustering:
        x = np.ascontiguousarray(x)
    dim_ = x.shape[1]
    kmeans = faiss.Clustering(x.shape[1], num_clusters)
    # initializing clusters with previous centroids
    if previous_centroids is not None:
        print(f'clustering with previous centroids..')
        kmeans.centroids.resize(previous_centroids.size)
        faiss.memcpy(kmeans.centroids.data(), faiss.swig_ptr(previous_centroids), previous_centroids.size * 4)
    kmeans.max_points_per_centroid = 10000000
    kmeans.niter = 100
    if previous_centroids is None:
        kmeans.nredo = 5
    else:
        kmeans.nredo = 1
    resources_ = faiss.StandardGpuResources()
    idx_config_ = faiss.GpuIndexFlatConfig()
    idx_config_.useFloat16 = False
    idx_config_.device = 0
    index = faiss.GpuIndexFlatL2(resources_, dim_, idx_config_)
    if nrns_clustering:
        x = np.ascontiguousarray(x)
    kmeans.train(x, index)
    centroids = faiss.vector_float_to_array(kmeans.centroids)
    objective = faiss.vector_float_to_array(kmeans.obj)
    centroids = centroids.reshape(num_clusters, dim_)
    index = faiss.IndexFlatL2(centroids.shape[1])
    index.add(centroids)
    distances, labels = index.search(x, 1)
    index.reset()
    resources_.noTempMemory()
    del kmeans
    del index
    del x
    gc.collect()
    return labels.ravel(), centroids


def get_nearest_neighbors(x, product_labels, k, queries=None):
    x = np.asarray(x.reshape(x.shape[0], -1), dtype=np.float32)
    with_queries = False
    if queries is None:
        with_queries = True
        queries = x
        k += 1
    dim_ = x.shape[1]
    resources_ = faiss.StandardGpuResources()
    idx_config_ = faiss.GpuIndexFlatConfig()
    idx_config_.useFloat16 = False
    idx_config_.device = 0
    index = faiss.GpuIndexFlatL2(resources_, dim_, idx_config_)
    index.add(x)
    _, neighbors = index.search(queries, k)
    if with_queries:
        for i in range(len(neighbors)):
            indices = np.nonzero(neighbors[i, :] != i)[0]
            indices.sort()
            if len(indices) > k - 1:
                indices = indices[:-1]
            neighbors[i, :-1] = neighbors[i, indices]
        neighbors = neighbors[:, :-1]
    n_neighbors = np.array([[product_labels[i] for i in ii] for ii in neighbors])
    index.reset()
    resources_.noTempMemory()
    del index
    del x
    gc.collect()
    return n_neighbors


def compute_embeddings(_model, dataloader, with_embedding_layer=True, with_norm=True):
    print('Computing embeddings..')
    start_time = time.time()
    _model.eval()
    assert not _model.training

    embeddings_set = [[] for _ in range(3)]
    all_embeddings = []
    with torch.no_grad():
        for b, batch in enumerate(dataloader):
            if b % 100 == 0:
                print(f'batch {b}/{len(dataloader)}')
            for ind, elems in enumerate(batch):
                if ind == 0:
                    if not with_embedding_layer:
                        elems, embeddings = _model(elems.to(device), with_embedding_layer)
                        all_embeddings.extend(embeddings.data.cpu().numpy())
                    else:
                        elems = _model(elems.to(device), with_embedding_layer)
                    if with_norm:
                        elems = f.normalize(elems, p=2, dim=1)
                    elems = elems.data.cpu().numpy()
                for elem in elems:
                    embeddings_set[ind].append(np.asarray(elem))
        result = [np.stack(embeddings_set[i]) for i in range(len(embeddings_set))]  # 3 x (samples_num x 2048)
    print(f'computing embeddings time: {round((time.time() - start_time) / 60, 3)}, min')
    if not with_embedding_layer:
        return result, all_embeddings
    return result


def cluster_embeddings(_model, dl, previous_centroids=None, is_finetune=False):
    if is_finetune:
        prod_labels = np.array(dl.dataset.labels)
        emb_indexes = np.array(dl.dataset.ids)
        cluster_labels_ = np.zeros(len(prod_labels), dtype=int)
        centroids, embeddings, final_embeddings = None, None, None
    else:
        (embeddings, prod_labels, emb_indexes), final_embeddings = compute_embeddings(_model, dl, with_embedding_layer=False,
                                                                                      with_norm=True)  # True
        print(f'emb shapes while clustering: {embeddings.shape}')
        sorted_lists = sort_(emb_indexes, [emb_indexes, embeddings, prod_labels])
        emb_indexes, embeddings, prod_labels = sorted_lists[0], sorted_lists[1], sorted_lists[2]
        start_time = time.time()
        cluster_labels_, centroids = kmeans(
            embeddings,
            num_clusters=nb_clusters,
            previous_centroids=previous_centroids)

        print(f'clustering time: {round((time.time() - start_time) / 60, 3)} min')
    return cluster_labels_, prod_labels, emb_indexes, centroids, embeddings, final_embeddings


def get_train_dl(idxs=None):
    if cur_dataset is 'OnlineProducts':
        ds_train_ = OnlineProductsDataset(ds_path=sop_dataset_path, mode='train', transforms_mode='train')
    elif cur_dataset is 'InShopClothes':
        ds_train_ = InShopDataset(ds_path=inshop_dataset_path, mode='train', transforms_mode='train')
    elif cur_dataset is 'CUB200':
        ds_train_ = CUBDataset(ds_path=cub_dataset_path, mode='train', transforms_mode='train')
    elif cur_dataset is 'CARS196':
        ds_train_ = CARSDataset(ds_path=cars_dataset_path, mode='train', transforms_mode='train')
    ds_train_.get_within_indexes(idxs)
    dl_train_ = torch.utils.data.DataLoader(ds_train_, batch_sampler=BatchSampler(ds_train_))
    return dl_train_

def get_optimal_nb_clusters_by_loss(embeddings, prod_labels, previous_centroids, writer, epoch, criterion, nb_clusters):
    start_time = time.time()
    losses = []
    distances_ap = []
    distances_an = []
    non_zeros_pos = []
    non_zeros_neg = []
    nb_clusters_list = []
    total_nrof_triplets = []
    total_neg_distances = []
    print('start validate nb_clusters...')
    for nb_clusters_ in range(8, 40):
        cluster_labels_, centroids = kmeans(
            embeddings,
            num_clusters=nb_clusters_,
            previous_centroids=previous_centroids)
        print(nb_clusters_)
        nb_clusters_list.append(nb_clusters_)
        # print(f'distance: {distance}')
        # writer.add_scalar('distance_%d'%nb_clusters_, distance, epoch)
        current_loss = []
        current_ap = []
        current_an = []
        current_non_zero_part_pos = []
        current_non_zero_part_neg = []
        current_nrof_triplets = []
        current_nrof_not_null_loss = []
        current_neg_dist = []
        for cluster_id in np.unique(cluster_labels_):
            indices = np.arange(len(embeddings),dtype=np.int)[cluster_labels_ == cluster_id]
            loss, d_ap, d_an, d_pn, non_zero_part_pos, non_zero_part_neg, nrof_triplets, total_neg_dist, nrof_not_null_loss = criterion(torch.tensor(embeddings[indices]).cuda(),
                                            torch.tensor(prod_labels[indices]).cuda(), epoch)
            if np.isnan(loss.item()):
                continue
            current_loss.append(loss.item())
            current_ap.append(d_ap.mean().item())
            current_an.append(d_an.mean().item())
            current_non_zero_part_pos.append(non_zero_part_pos.item())
            current_non_zero_part_neg.append(non_zero_part_neg.item())
            current_nrof_triplets.append(nrof_triplets)
            current_neg_dist.append(total_neg_dist.item())
            current_nrof_not_null_loss.append(nrof_not_null_loss)
            print(current_loss[-1], current_ap[-1], current_an[-1])
        losses.append(np.mean(current_loss))
        distances_ap.append(np.mean(current_ap))
        distances_an.append(np.mean(current_an))
        non_zeros_pos.append(np.mean(current_non_zero_part_pos))
        non_zeros_neg.append(np.mean(current_non_zero_part_neg))
        total_nrof_triplets.append(np.mean(nrof_triplets))
        total_neg_distances.append(np.mean(current_neg_dist))
        writer.add_scalar('loss/epoch_%d' % epoch, np.mean(current_loss), nb_clusters_)
        writer.add_scalar('d_ap/epoch_%d' % epoch, np.mean(current_ap), nb_clusters_)
        writer.add_scalar('d_an/epoch_%d' % epoch, np.mean(current_an), nb_clusters_)
        writer.add_scalar('non_zero_part_pos/epoch_%d' % epoch, np.mean(current_non_zero_part_pos), nb_clusters_)
        writer.add_scalar('non_zero_part_neg/epoch_%d' % epoch, np.mean(current_non_zero_part_neg), nb_clusters_)
        writer.add_scalar('nrof_triplets/epoch_%d' % epoch, np.mean(nrof_triplets), nb_clusters_)
        writer.add_scalar('current_neg_dist/epoch_%d' % epoch, np.mean(current_neg_dist), nb_clusters_)
        writer.add_scalar('current_nrof_not_null_loss/epoch_%d' % epoch, np.mean(current_nrof_not_null_loss), nb_clusters_)
    plt.cla()
    plt.plot(nb_clusters_list, losses, label='losses', color='r')
    plt.plot(nb_clusters_list, non_zeros_pos, label='non_zeros_pos', color='b')
    plt.plot(nb_clusters_list, non_zeros_neg, label='non_zeros_neg', color='g')
    # plt.plot(nb_clusters_list, total_neg_distances, label='total_neg_distances', color='y')
    plt.legend()
    plt.savefig(tensorboard_dir+'different_nb_clusters_%d.png'%epoch)
    plt.cla()
    plt.plot(nb_clusters_list, total_nrof_triplets, label='total_nrof_triplets', color='b')
    plt.savefig(tensorboard_dir+'total_nrof_triplets_different_nb_clusters_%d.png'%epoch)
    # plt.show()
    plt.cla()
    print(f'clustering time: {round((time.time() - start_time) / 60, 3)} min')
    best_index = int(np.argmax(losses))
    nb_clusters_ = nb_clusters_list[best_index]
    gc.collect()
    print('optimal nb_clusters: %d, loss: %.3f, d_ap: %.3f, d_an: %.3f'% (nb_clusters_, losses[best_index],
                                                                          distances_ap[best_index],
                                                                          distances_an[best_index]))

def sort_(indexes_, lists_to_sort):
    sorted_indexes = np.sort(indexes_)
    sorted_lists = [list_[sorted_indexes] for list_ in lists_to_sort]
    return sorted_lists


def round_neurons_num(neurons_percents):
    def err(p, rounded_p):
        d = np.sqrt(1.0 if p < 1.0 else p)
        return abs(rounded_p - p) ** 2 / d

    if not np.isclose(sum(neurons_percents), embedding_dim):
        print(f'neurons_percents: {neurons_percents}')
        raise ValueError
    n = len(neurons_percents)
    rounded = [int(x) for x in neurons_percents]
    dif = embedding_dim - sum(rounded)
    errs = [(err(neurons_percents[i], rounded[i] + 1) - err(neurons_percents[i], rounded[i]), i) for i in range(n)]
    r = sorted(errs)
    for i in range(dif):
        rounded[r[i][1]] += 1
    return rounded


def init_learners_masks(cluster_labels_, product_labels_, embeddings_):
    print(f'initializing learners masks...')
    st = time.time()
    neurons_nmis = np.zeros((embedding_dim, nb_clusters))
    for cluster_id in range(nb_clusters):
        ids = np.where(cluster_labels_ == np.repeat(cluster_id, len(cluster_labels_)))[0]
        cl_labels = cluster_labels_[ids]
        prod_labels = product_labels_[ids]
        prod_labels_count = len(np.unique(prod_labels))
        embeds = embeddings_[ids]
        assert len(embeds) == len(prod_labels)
        neurons_outs = np.split(embeds, embedding_dim, axis=1)

        for i, nrn_outs in enumerate(neurons_outs):
            assert len(nrn_outs) == len(prod_labels)
            nrn_cluster_lbls, _ = kmeans(nrn_outs, num_clusters=prod_labels_count, nrns_clustering=True)
            assert len(nrn_cluster_lbls) == len(prod_labels)
            nmi = nmi_(prod_labels, nrn_cluster_lbls)
            neurons_nmis[i][cluster_id] = nmi
    print(f'masks calculating time: {round((time.time() - st)/60, 3)} min')
    return neurons_nmis


def calculate_neurons_nmi(cluster_labels_, product_labels_, embeddings_, epoch):
    print(f'assigning neurons with nmi + lsa...')
    st = time.time()
    result = {}
    embeddings_ = np.asarray(embeddings_)
    for cluster_id in range(nb_clusters):
        result[cluster_id] = {}
        ids = np.where(cluster_labels_ == np.repeat(cluster_id, len(cluster_labels_)))[0]
        # cl_labels = cluster_labels_[ids]
        prod_labels = product_labels_[ids]
        prod_labels_count = len(np.unique(prod_labels))
        embeds = embeddings_[ids]
        assert len(embeds) == len(prod_labels)

        nrn_cluster_lbls, _ = kmeans(embeds, num_clusters=prod_labels_count, nrns_clustering=True)
        assert len(nrn_cluster_lbls) == len(prod_labels)
        nmi = nmi_(prod_labels, nrn_cluster_lbls)
        result[cluster_id]['all_embedding'] = nmi
        result[cluster_id]['exclude'] = {}
        for i in range(embedding_dim):
            # assert len(nrn_outs) == len(prod_labels)
            nrn_outs = embeds[:, np.arange(embedding_dim)!= i]
            assert len(nrn_outs[0]) == (embedding_dim-1)
            nrn_cluster_lbls, _ = kmeans(nrn_outs, num_clusters=prod_labels_count, nrns_clustering=True)
            assert len(nrn_cluster_lbls) == len(prod_labels)
            nmi = nmi_(prod_labels, nrn_cluster_lbls)
            result[cluster_id]['exclude'][i] = nmi
        gc.collect()
    with open(tensorboard_dir+'neurons_nmis_%d.json'%epoch, 'w') as file:
        json.dump(result, file)
    del result


def crazy_stuff(_model_,
                dataloader_train,
                writer,
                epoch,
                criterion,
                is_reclustering_epoch=False,
                previous_indexes=None,
                previous_cluster_labels=None,
                previous_centroids=None,
                is_fine_tune=False,
                is_fine_tune1=False):
    
    
    if is_reclustering_epoch:
        cluster_labels_, product_labels_, indexes_, centroids, embeddings, final_embeddings = cluster_embeddings(_model_,
                                                                                               dataloader_train,
                                                                                               previous_centroids=previous_centroids,
                                                                                               is_finetune=is_fine_tune1)
    else:
        cluster_labels_, product_labels_, indexes_, centroids, embeddings, final_embeddings = cluster_embeddings(_model_,
                                                                                               dataloader_train,
                                                                                               is_finetune=is_fine_tune1)
#     with torch.no_grad():
#         get_optimal_nb_clusters_by_loss(np.asarray(final_embeddings), product_labels_, previous_centroids, writer, epoch, criterion, nb_clusters)
    sorted_list = sort_(indexes_, [indexes_, product_labels_, cluster_labels_, embeddings])
    indexes_, product_labels_, cluster_labels_, embeddings = sorted_list[0], sorted_list[1], sorted_list[2], \
                                                             sorted_list[3]

    nrof_reassign_images = 0
    for c in range(nb_clusters):
        for t in np.unique(product_labels_[cluster_labels_ == c]):
            condition = product_labels_[cluster_labels_ == c] == t
            if (condition).sum().item() == 1:
                if np.sum(product_labels_ == t).sum().item() == 1:
                    cluster_labels_[(product_labels_ == t) & (cluster_labels_ == c)] = -1
                    continue
                nrof_reassign_images+=1
                embedding = embeddings[cluster_labels_ == c, :][condition, :]
                dist_to_clusters = np.linalg.norm(embedding-centroids, axis=1)
                product_clusters = np.unique(cluster_labels_[product_labels_ == t])
                dist_to_clusters[c] = 10e7
                dist_to_clusters[np.logical_not(np.isin(np.arange(nb_clusters), product_clusters))] = 10e7
                assert np.any(dist_to_clusters<10e6)
                new_c = np.argmin(dist_to_clusters)
                cluster_labels_[(product_labels_ == t) & (cluster_labels_ == c)] = new_c

    log_metrics('clusters/nrof_unused_images', np.sum(cluster_labels_ == -1), epoch)        

    # if not (is_fine_tune or is_fine_tune1) and epoch%10 == 0:
    #     calculate_neurons_nmi(cluster_labels_, product_labels_, final_embeddings, epoch)
    
    # get NMI and loss of clusters
    current_loss = [] 
    t1 = time.time()
    for c in np.unique(cluster_labels_):
        indices = np.arange(len(final_embeddings),dtype=np.int)[cluster_labels_ == c]
        emb = torch.tensor([final_embeddings[ind] for ind in indices]).cuda()
        prod_labels = torch.tensor([product_labels_[ind] for ind in indices]).cuda()
        loss = criterion(emb, prod_labels, epoch, for_cluster=True)
        if np.isnan(loss.item()):
            continue
        current_loss.append(loss.item())
        cl_lbls, _ = kmeans(emb.cpu().numpy(), num_clusters=len(np.unique(prod_labels.cpu().numpy())))
        cl_nmi = nmi_(cl_lbls, prod_labels.cpu().numpy()) * 100
        log_metrics(f'clusters/cluster_{c}_nmi', cl_nmi, epoch)
        log_metrics(f'clusters/cluster_{c}_loss', np.mean(current_loss), epoch)
        print(f'cluster: {c}, nmi: {cl_nmi}, loss: {loss}')
    print(f'getting loss and nmi for clusters time: {(time.time() - t1)/60} min')
    
    # get num_of_imgs_in_dif_clusters
    nb_imgs_in_other_clusters = []
    for i, pr_lbl in enumerate(np.unique(product_labels_)):
        clusters_ids = cluster_labels_[product_labels_ == pr_lbl]
        cnt = Counter(clusters_ids)
        most_common_cluster, nb_imgs = cnt.most_common(1)[0]
        count_in_other_clusters = np.sum(list(cnt.values())) - nb_imgs
        nb_imgs_in_other_clusters.append(count_in_other_clusters)
    part_imgs_from_other_clusters = np.sum(nb_imgs_in_other_clusters) / len(product_labels_)
    print(f'part_imgs_from_other_clusters: {part_imgs_from_other_clusters}')
    log_metrics(f'clusters/part_imgs_from_other_clusters1', part_imgs_from_other_clusters, epoch)
    
    dls = [[] for _ in range(nb_clusters)]
    for cluster_id in range(nb_clusters):
        idxs = indexes_[cluster_labels_ == cluster_id]
        dls[cluster_id] = get_train_dl(idxs=idxs)
    return dls, cluster_labels_, product_labels_, indexes_, centroids


def make_training_step(batch, learner_id, model, criterion, optimizer, iter_, epoch, cross_entropy=None, is_fine_tune=False):
    images, product_labels = batch[0].cuda(), batch[1].cuda()
    optimizer.zero_grad()
    
    if cross_entropy is not None:
        embeddings, logits = model(images, with_softmax=True)
        cls_loss = cross_entropy(logits, product_labels)*0.02
        log_metrics('loss/cls_loss', cls_loss.item(), iter_)
    else:
        embeddings = model(images)
        cls_loss = 0

    if not is_fine_tune:
        current_mask = model.masks(torch.tensor(learner_id).cuda())
        current_mask = torch.nn.functional.relu(current_mask)
        l1_regularization = lambda1 * torch.norm(current_mask, p=1)/embeddings.size(0)
        l1_regularization = l1_regularization.cuda()
        log_metrics('masks/nrof_positive_mask_%d'%learner_id, torch.nonzero(current_mask).size(0), iter_)
        log_metrics('masks/mean_mask_%d'%learner_id, current_mask.mean().item(), iter_)
        l2_regularization = lambda2 * embeddings.norm(2)/np.sqrt(embeddings.size(0))
        embeddings = embeddings * current_mask
    else:
        l1_regularization = 0
        l2_regularization = 0

    l2_regularization = l2_regularization + lambda2 * embeddings.norm(2)/np.sqrt(embeddings.size(0))
    l2_regularization = l2_regularization.cuda()
        # embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    loss, d_ap, d_an, d_pn, non_zero_part_pos, non_zero_part_neg, nrof_triplets, total_neg_dist, nrof_not_null_loss, beta, unused_neg_part = \
        criterion(embeddings, product_labels, iter_, learner_id=learner_id)
    loss = loss + l1_regularization + l2_regularization+cls_loss
    if not is_fine_tune:
        log_metrics('loss/L1_masks_reg', l1_regularization.item(), iter_)
        log_metrics('loss/L1_masks_reg_learner_id_%d'%learner_id, l1_regularization.item(), iter_)
    log_metrics('loss/L2_emb_reg', l2_regularization.item(), iter_)
    log_metrics('loss/L2_emb_reg_learner_id_%d'%learner_id, l2_regularization.item(), iter_)
    log_metrics('loss/total_loss', loss.item(), iter_)
    log_metrics('distances/d_ap', d_ap.mean().item(), iter_)
    log_metrics('distances/d_ap_learner_id_%d'%learner_id, d_ap.mean().item(), iter_)
    log_metrics('distances/d_an', d_an.mean().item(), iter_)
    log_metrics('distances/d_an_learner_id_%d'%learner_id, d_an.mean().item(), iter_)
    if not torch.isnan(loss).any():
        loss.backward()
#         with amp.scale_loss(loss, optimizer) as scaled_loss:
#             scaled_loss.backward()
        optimizer.step()

        with open(f'masks_epoch_{epoch}.json', 'w') as out_file:
            masks_ = model.masks.weight.cpu().detach().numpy().tolist()
            json.dump({'masks': masks_}, out_file)
        log_artifacts(f'/kaggle/working/masks_epoch_{epoch}.json', f'masks_epoch_{epoch}.json')
        
        return loss.item(), d_ap, d_an, d_pn, non_zero_part_pos.item(), non_zero_part_neg.item(), nrof_triplets, total_neg_dist, nrof_not_null_loss, beta, unused_neg_part
    else:
        print(f'Loss is nan')
        return None, d_ap, d_an, d_pn, non_zero_part_pos.item(), non_zero_part_neg.item(), nrof_triplets, total_neg_dist, nrof_not_null_loss, beta, unused_neg_part


def get_recall_at_k(product_labels, neighbours_ids, cur_k):
    sum_ = 0
    for query_lbl, nearest_ids in zip(product_labels, neighbours_ids):
        if query_lbl in nearest_ids[:cur_k]:
            sum_ += 1
    recall_at_k = sum_ / len(product_labels)
    return recall_at_k


def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)


def eval_func_isc(query_embeddings, gallery_embeddings, query_product_labels, gallery_product_labels, k_list, nb_classes, writer, epoch, with_norm, use_mask):
    if with_norm:
        query_embeddings = normalized(np.copy(query_embeddings))
        gallery_embeddings = normalized(np.copy(gallery_embeddings))
    neighbours = get_nearest_neighbors(gallery_embeddings, gallery_product_labels, max(k_list),
                                       queries=query_embeddings)
    product_labels_all = np.concatenate([query_product_labels, gallery_product_labels])
    embeddings_all = np.concatenate([query_embeddings, gallery_embeddings])
    clusters_labels, _ = kmeans(embeddings_all, num_clusters=nb_classes)
    nmi = nmi_(clusters_labels, product_labels_all) * 100
    print(f'nmi: {nmi}')
    out = []
    gc.collect()
    torch.cuda.empty_cache()
    for k in k_list:
        recall_at_k = get_recall_at_k(query_product_labels, neighbours, k) * 100
        recall_metrics_name = f'Recall_at_{k}_w_norm' if with_norm else f'Recall_at_{k}_not_norm'
        print(f'{recall_metrics_name}: {recall_at_k}')
        if writer is not None:
            writer.add_scalar(recall_metrics_name, recall_at_k, epoch)
        log_metrics(f'eval/{recall_metrics_name}', recall_at_k, epoch)
        out.append(recall_at_k)
    out.append(nmi)
    nmi_metrics_name = 'NMI_w_norm' if with_norm else 'NMI_not_norm'
    if writer is not None:
        writer.add_scalar(nmi_metrics_name, nmi, epoch)
    log_metrics(f'eval/{nmi_metrics_name}', nmi, epoch)
    str_ = ' '.join([str(elem) for elem in out])
    with open(os.path.join(metrics_dir, f'metrics'), 'a') as f:
        f.write(f'epoch_{epoch} norm {with_norm} ' + str_ + '\n')
    return out


def evaluate_isc(writer, epoch, is_fine_tune=False, with_norm=True, mode=None):  # for InShopClothes dataset
    query_embeddings, query_product_labels, _ = compute_embeddings(model_, dl_query, with_norm=False)
    gallery_embeddings, gallery_product_labels, _ = compute_embeddings(model_, dl_gallery, with_norm=False)    
    
    print(f'emb shape while evaluating: {query_embeddings.shape}, {gallery_embeddings.shape}')
    nb_classes = dl_query.dataset.get_nb_classes()
    assert nb_classes == len(set(query_product_labels))
    original_query_embeddings = np.copy(query_embeddings)
    original_gallery_embeddings = np.copy(gallery_embeddings)
    k_list = [1, 10, 20, 30, 50]
    for use_mask in ['not']:# + list(np.arange(8)):
        for with_norm in [False, True]:
            if use_mask == 'not':
                query_embeddings = original_query_embeddings
                gallery_embeddings = original_gallery_embeddings
#             else:
#                 query_embeddings = np.copy(original_query_embeddings) * torch.nn.functional.relu(model_.masks.weight[use_mask]).detach().cpu().numpy()
#                 gallery_embeddings = np.copy(original_gallery_embeddings) * torch.nn.functional.relu(model_.masks.weight[use_mask]).detach().cpu().numpy()
                                
            out = eval_func_isc(query_embeddings, gallery_embeddings, query_product_labels, gallery_product_labels, k_list, nb_classes, writer, epoch, with_norm, use_mask)
        gc.collect()
    return out


def eval_func(embeddings, product_labels, k_list, nb_classes, writer, epoch, with_norm, use_mask):
    if with_norm:
        embeddings = normalized(np.copy(embeddings))
    neighbours = get_nearest_neighbors(embeddings, product_labels, max(k_list))
    clusters_labels, _ = kmeans(embeddings, num_clusters = nb_classes)
    nmi = nmi_(clusters_labels, product_labels) * 100
    print(f'nmi: {nmi}')
    out = []
    gc.collect()
    torch.cuda.empty_cache()
    for k in k_list:
        recall_at_k = get_recall_at_k(product_labels, neighbours, k) * 100
        recall_metrics_name = f'Recall_at_{k}_w_norm' if with_norm else f'Recall_at_{k}_not_norm'
        print(f'{recall_metrics_name}: {recall_at_k}')
        if writer is not None:
            writer.add_scalar(recall_metrics_name, recall_at_k, epoch)
        log_metrics(f'eval/{recall_metrics_name}', recall_at_k, epoch)
        out.append(recall_at_k)
    out.append(nmi)
    nmi_metrics_name = 'NMI_w_norm' if with_norm else 'NMI_not_norm'
    if writer is not None:
        writer.add_scalar(nmi_metrics_name, nmi, epoch)
    log_metrics(f'eval/{nmi_metrics_name}', nmi, epoch)
    str_ = ' '.join([str(elem) for elem in out])
    with open(os.path.join(metrics_dir, f'metrics'), 'a') as f:
        f.write(f'epoch_{epoch} norm {with_norm} ' + str_ + '\n')
    return out


def evaluate(writer, epoch, dl_, is_fine_tune=False, with_norm=True, mode='test'):
    # compute query images embeddings = retrieval images embeddings
    print(f'Evaluating on {mode} data')
    embeddings, product_labels, _ = compute_embeddings(model_, dl_, with_norm=False)
    print(f'emb shape while evaluating: {embeddings.shape}')
    nb_classes = dl_test.dataset.get_nb_classes()
    original_embeddings = np.copy(embeddings)
    if cur_dataset is 'CUB200' or cur_dataset is 'CARS196':
        k_list = [1, 2, 4, 8]
    elif cur_dataset is 'OnlineProducts': 
        k_list = [1, 10, 100, 1000]
        
    for use_mask in ['not']: # + list(np.arange(8)):
        for with_norm in [False, True]:
            if use_mask == 'not':
                embeddings = original_embeddings
#             else:
#                 embeddings = np.copy(original_embeddings) * torch.nn.functional.relu(model_.masks.weight[use_mask]).detach().cpu().numpy()
            out = eval_func(embeddings, product_labels, k_list, nb_classes, writer, epoch, with_norm, use_mask)
        gc.collect()
    return out


def get_gpu_memory():
    gpu_resources = faiss.StandardGpuResources()
    idx_config = faiss.GpuIndexFlatConfig()
    idx_config.useFloat16 = False
    idx_config.device = 0
    index = faiss.GpuIndexFlatL2(gpu_resources, 2048, idx_config)
    return index, gpu_resources


def save_all_data_to_tar(tarfile_name, cur_epoch):
    try:
        all_files = os.listdir('/kaggle/working/')
        tar = tarfile.open(f"{tarfile_name}.tar.gz", "w:gz")
        for item in all_files:
            if not item.startswith('all') and f'_{cur_epoch}.' in item or f'_{cur_epoch - 1}.' in item or item.startswith('metrics'):
                tar.add(item)
        tar.close()
    except:
        print('error while writing to tarfile')

        
class LabelSmoothLoss(torch.nn.Module):
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, input, target):
        log_prob = torch.nn.functional.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
                 self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss

def train(model, criterion, optimizer, all_betas):
    if cur_dataset is 'OnlineProducts':
        ds_train = OnlineProductsDataset(ds_path=sop_dataset_path, mode='train')
    elif cur_dataset is 'InShopClothes':
        ds_train = InShopDataset(ds_path=inshop_dataset_path, mode='train')
    elif cur_dataset is 'CUB200':
        ds_train = CUBDataset(ds_path=cub_dataset_path, mode='train')
    elif cur_dataset is 'CARS196':
        ds_train = CARSDataset(ds_path=cars_dataset_path, mode='train')
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size)

    dl_train_list, cluster_labels, product_labels, indexes = None, None, None, None
    is_fine_tune_epoch, is_fine_tune_epoch1 = False, False
    start_epoch, global_step = 0, -1
    
#     if use_mean_beta_at_finetune:
#         criterion[-1].beta.data = torch.tensor([torch.mean(torch.tensor([crit.beta.data for i, crit in enumerate(criterion[:-1])]))]).cuda()

    if use_lr_scheduler:
        num_steps = 200000
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps, eta_min=5e-7)
        warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=100,
                                                  after_scheduler=lr_scheduler)

    if continue_training_from_epoch:
        try:
            checkpoint = torch.load(os.path.join(input_dir, f'checkpoint_{checkpoint_from_epoch}.pth'))
            model.load_state_dict(checkpoint['model'])
            start_epoch = checkpoint['epoch'] + 1
            global_step = checkpoint['global_step'] + 1
            optimizer.load_state_dict(checkpoint['opt'])
            for state in optimizer.state.values():
                for k, v in state.items():
                    if isinstance(v, torch.Tensor):
                        state[k] = v.to(device)
#             amp.load_state_dict(checkpoint['amp_state_dict'])
            for i, crit in enumerate(criterion):
                crit.beta.data=checkpoint['beta'][i]
            print(f'Loading model saved in epoch: {start_epoch - 1}')
        except FileNotFoundError:
            print('Checkpoint not found')

        print('Trying to load saved clusters..')
        try:
            clusters_checkpoint = torch.load(os.path.join(input_dir, f'clusters_at_epoch_{clusters_from_epoch}.pth'))
            cl_epoch = clusters_checkpoint['epoch'] + 1
            dl_train_list = clusters_checkpoint['dl_train_list']
            cluster_labels = clusters_checkpoint['cluster_labels']
            indexes = clusters_checkpoint['indexes']
            centroids = clusters_checkpoint['centroids']
            print(f'Loading clusters saved in epoch: {cl_epoch - 1}')
        except FileNotFoundError:
            print('Clusters checkpoint not found')
    
    # model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    cross_entropy = LabelSmoothLoss(0.1) if use_cls_loss else None  
    optimizer.zero_grad()
    optimizer.step()
    if compute_metrics_before_training:
        model.eval()
        assert not model.training
        with torch.no_grad():
            if cur_dataset is 'InShopClothes':
                metrics = evaluate_isc(None, -1)
            else:
                metrics = evaluate(None, -1, dl_=dl_test)
        model.train()
        assert model.training

    for e in range(start_epoch, start_epoch + epochs):
        if e >= ft_epoch:
            is_fine_tune_epoch = True
        if e >= fine_tune_epoch:
            is_fine_tune_epoch1 = True
        print(f'Epoch: {e}')
        epoch_start_time = time.time()
#         writer = SummaryWriter(log_dir= tensorboard_dir + f'/epoch_{e}')
        writer = None

        if e % recluster_epoch_freq == 0:
            model.eval()
            assert not model.training
            print('Clustering..')
            with torch.no_grad():
                if e == 0:
                    dl_train_list, cluster_labels, product_labels, indexes, centroids = \
                        crazy_stuff(model, dl_train, writer, e, criterion[-1], is_fine_tune=is_fine_tune_epoch,
                                    is_fine_tune1=is_fine_tune_epoch1)
                else:
                    dl_train_list, cluster_labels, product_labels, indexes, centroids = \
                        crazy_stuff(model, dl_train, writer, e, criterion[-1], is_reclustering_epoch=True, previous_indexes=indexes,
                                    previous_cluster_labels=cluster_labels, previous_centroids=centroids,
                                    is_fine_tune=is_fine_tune_epoch, is_fine_tune1=is_fine_tune_epoch1)
            for cluster_id in range(nb_clusters):
                if writer is not None:
                    writer.add_scalar('nb_images/cluster_%d'%cluster_id, np.sum(cluster_labels == cluster_id), e)
                log_metrics('clusters/nb_images_cluster_%d'%cluster_id, np.sum(cluster_labels == cluster_id), e)
            if save_clusters:
                print('Saving clusters...')
                clusters_checkpoint = {
                    'epoch': e,
                    'dl_train_list': dl_train_list,
                    'cluster_labels': cluster_labels,
                    'product_labels': product_labels,
                    'indexes': indexes,
                    'centroids': centroids
                }
                torch.save(clusters_checkpoint,
                           (os.path.join(checkpoints_dir, f'clusters_at_epoch_{e}.pth')))

        model.train()
        assert model.training
        print('Starting training..')
        nb_batches = [len(dl) for dl in dl_train_list]
        print('nb_bathes in each dl', nb_batches)
        for nb_b in nb_batches:
            log_metrics('clusters/nb_batches', nb_b, e)
        if cur_dataset is 'CUB200' or cur_dataset is 'CARS196':
            _nb_batches_ = min(nb_batches)
        else:
            _nb_batches_ = int(np.mean(nb_batches))  # max(nb_batches)
        loss_list, d_ap_list, d_an_list, d_pn_list, beta_list, unused_neg_part_list = list(), list(), list(), list(), list(), list()
        loss_by_learners, d_ap_by_learners, d_an_by_learners, d_pn_by_learners, beta_by_learners = {i:[] for i in range(nb_clusters)}, {i:[] for i in range(nb_clusters)}, {i:[] for i in range(nb_clusters)}, {i:[] for i in range(nb_clusters)}, {i:[] for i in range(nb_clusters)}
        unused_neg_part_by_learners = {i:[] for i in range(nb_clusters)}
        nb_iters = _nb_batches_ * nb_clusters
        print(f'Iterations num: {nb_iters}')
        log_metrics('clusters/Iterations num', nb_iters, e)
        with open(os.path.join(metrics_dir, f'model.masks'), 'a') as f:
            f.write(f'epoch_{e} ' + str(model.masks) + '\n')
        
        gc.collect()
        for j in range(_nb_batches_):
            for i, dl in enumerate(dl_train_list):
                if len(dl) != 0:
                    dl = iter(dl)
                    batch = next(dl)
                    if batch is None:
                        dl = iter(dl)
                        batch = next(dl)
                    learner_id = i
                    loss_id = -1 if is_fine_tune_epoch else i
                    
                    with torch.autograd.set_detect_anomaly(True):
                        loss_on_batch, d_ap, d_an, d_pn, non_zero_part_pos, non_zero_part_neg, \
                        nrof_triplets, total_neg_dist, nrof_not_null_loss, beta, unused_neg_part = make_training_step(batch, learner_id, model, criterion[loss_id], optimizer,
                                                                       global_step, e, cross_entropy=cross_entropy,
                                                                       is_fine_tune=is_fine_tune_epoch)
                        if loss_on_batch != None:
                            global_step += 1
    #                         if writer is not None:
                            log_metrics('loss/Loss_train', loss_on_batch, global_step)
                            log_metrics(f'loss/_loss_learner_{learner_id}', loss_on_batch, global_step)
                            log_metrics(f'distances/d_ap_learner_{learner_id}', d_ap.mean().item(), global_step)
                            log_metrics(f'distances/d_an_learner_{learner_id}', d_an.mean().item(), global_step)
                            log_metrics(f'distances/d_pn_learner_{learner_id}', d_pn.mean().item(), global_step)

                            log_metrics('loss/non_zero_part_pos_loss', non_zero_part_pos, global_step)
                            log_metrics(f'loss/non_zero_part_pos_loss_learner_{learner_id}', loss_on_batch, global_step)

                            log_metrics('loss/non_zero_part_neg_loss', non_zero_part_neg, global_step)
                            log_metrics(f'loss/non_zero_part_neg_loss_learner_{learner_id}', non_zero_part_neg, global_step)

                            log_metrics('loss/nrof_not_null_loss', nrof_not_null_loss, global_step)
                            log_metrics(f'loss/nrof_not_null_loss_learner_{learner_id}', nrof_not_null_loss, global_step)

                            log_metrics(f'clusters/nrof_triplets_learner_{learner_id}', nrof_triplets, global_step)
                            log_metrics(f'clusters/nrof_triplets_all', nrof_triplets, global_step)
                            log_metrics(f'distances/total_neg_dist_learner_{learner_id}', total_neg_dist.item(), global_step)
                            log_metrics(f'distances/total_neg_dist_all', total_neg_dist.item(), global_step)
                            for i in range(len(criterion_)):
                                log_metrics(f'loss/beta_learner_id_{i}', criterion_[i].beta[0].item(), global_step)

                            loss_list.append(loss_on_batch)
                            loss_by_learners[learner_id].append(loss_on_batch)
                            d_ap_by_learners[learner_id].append(d_ap.data.cpu().numpy().tolist())
                            d_an_by_learners[learner_id].append(d_an.data.cpu().numpy().tolist())
                            d_pn_by_learners[learner_id].append(d_pn.data.cpu().numpy().tolist())
                            unused_neg_part_by_learners[learner_id].append(unused_neg_part)
                            beta_by_learners[learner_id].append(beta)
                            
                            d_ap_list.append(d_ap.data.cpu().numpy().tolist())
                            d_an_list.append(d_an.data.cpu().numpy().tolist())
                            d_pn_list.append(d_pn.data.cpu().numpy().tolist())
                            beta_list.append(beta)
                            unused_neg_part_list.append(unused_neg_part)

                            if global_step % 50 == 0:
                                if global_step != 0:
                                    loss_mean = np.mean(loss_list[-50:])
                                else:
                                    loss_mean = loss_on_batch     
                                print(f'global step: {global_step}, loss: {loss_mean}')
            
            if use_lr_scheduler:
                lr_scheduler.step(global_step)
                warmup_scheduler.step(global_step)

                for param_group in optimizer.param_groups:
                    if writer is not None:
                        writer.add_scalar('lr', param_group['lr'], global_step)
                    log_metrics('loss/lr', param_group['lr'], global_step)
                    continue
                
        log_metrics('distances/d_ap_mean', np.mean(d_ap_list), e)
        log_metrics('distances/d_an_mean', np.mean(d_an_list), e)
        log_metrics('distances/d_pn_mean', np.mean(d_pn_list), e)
        log_metrics('loss/mean_loss', np.mean(loss_list), e)
        log_metrics('loss/mean_beta', np.mean(beta_list), e)
        log_metrics('clusters/mean_unused_w_cutoff_neg_part', np.mean(unused_neg_part_list), e)
        
        for learner_id in range(nb_clusters):
            log_metrics('loss/mean_loss_learner_id_%d'%learner_id, np.mean(loss_by_learners[learner_id]), e)
            log_metrics('distances/mean_d_ap_learner_id_%d'%learner_id, np.mean(d_ap_by_learners[learner_id]), e)
            log_metrics('distances/mean_d_an_learner_id_%d'%learner_id, np.mean(d_an_by_learners[learner_id]), e)
            log_metrics('distances/mean_d_pn_learner_id_%d'%learner_id, np.mean(d_pn_by_learners[learner_id]), e)
            log_metrics('loss/mean_beta_learner_id_%d'%learner_id, np.mean(beta_by_learners[learner_id]), e)
            log_metrics('clusters/mean_unused_w_cutoff_neg_part_learner_id_%d'%learner_id, np.mean(unused_neg_part_by_learners[learner_id]), e)
        
        if to_make_density_plots and e % 2 == 0 and not is_fine_tune_epoch1:
            print(f'making density plots..')
            d_ap_list_ = np.concatenate(d_ap_list, 0)
            d_an_list_ = np.concatenate(d_an_list, 0)
            make_density_plots(e, d_ap_list_, d_an_list_, mode='train')

        print(f'epoch training time: {round((time.time() - epoch_start_time) / 60, 3)} min')
        eval_start_time = time.time()
        model.eval()
        assert not model.training
        with torch.no_grad():
            if cur_dataset is 'InShopClothes':
                metrics = evaluate_isc(writer, e)
            else:
                metrics = evaluate(writer, e, dl_=dl_test)
                if evalaute_on_train_data:
                    metrics = evaluate(writer, e, dl_=dl_train, mode='train')
                    
        if use_mean_beta_at_finetune and e == ft_epoch - 1:
            criterion[-1].beta.data = torch.tensor([torch.mean(torch.tensor([crit.beta.data for i, crit in enumerate(criterion[:-1])]))]).cuda()   
#             print('beta at finetune start', criterion[-1].beta.data)
        
        if save_model:
            print('Saving current model...')
            state = {
                'model': model.state_dict(),
#                 'amp_state_dict': amp.state_dict(),
                'epoch': e,
                'global_step': global_step,
                'opt': optimizer.state_dict(),
                'beta': [criterion_[i].beta for i in range(len(criterion_))]
            }
            torch.save(state, (os.path.join(checkpoints_dir, f'checkpoint_{e}.pth')))
            
        files_to_save = os.listdir('/kaggle/working/')
        tar = tarfile.open(f'mlruns_epoch_{e}.tar.gz', 'w:gz')
        tar1 = tarfile.open(f'all_metrics_{e}.tar.gz', 'w:gz')
        for item in files_to_save:
            if item.startswith('mlruns') and not item.startswith('mlruns_epoch'):
                tar.add(item)
            if item in dirs_for_logs:
                tar1.add(item)
        tar.close()
        tar1.close()
        
#         save_all_data_to_tar(f'all_epoch_{e}', e)
        
        if os.path.exists(os.path.join(checkpoints_dir, f'checkpoint_{e-5}.pth')):
            os.remove(os.path.join(checkpoints_dir, f'checkpoint_{e-5}.pth'))
            
        if os.path.exists(os.path.join(checkpoints_dir, f'clusters_at_epoch_{e-10}.pth')):
            os.remove(os.path.join(checkpoints_dir, f'clusters_at_epoch_{e-10}.pth'))
            
        if os.path.exists(os.path.join(checkpoints_dir, f'mlruns_epoch_{e-5}.tar.gz')):
            os.remove(os.path.join(checkpoints_dir, f'mlruns_epoch_{e-5}.tar.gz'))
            
#         if os.path.exists(os.path.join(checkpoints_dir, f'all_epoch_{e-2}.tar.gz')):
#             os.remove(os.path.join(checkpoints_dir, f'all_epoch_{e-2}.tar.gz'))
            
        print(f'validation time: {round((time.time() - eval_start_time) / 60, 3)} min')
        print(f'Epoch total time: {round((time.time() - epoch_start_time) / 60, 3)} min')

def test(model, epoch):
    print(f'testing model..')
    if load_saved_model_for_test:
        try:
            checkpoint = torch.load(os.path.join(input_dir, f'checkpoint_{epoch}.pth'))
            model.load_state_dict(checkpoint['model'])
            cur_epoch = checkpoint['epoch']
            print(f'Loading model saved in epoch: {cur_epoch}')
        except FileNotFoundError:
            print('Checkpoint not found')
    model.eval()
    assert not model.training
    with torch.no_grad():
        if cur_dataset is 'InShopClothes':
            metrics = evaluate_isc(None, -1)
        else:
            metrics = evaluate(None, -1)


if __name__ == '__main__':
    print(f'selected dataset: {cur_dataset}')
    torch.cuda.set_device(0)
    np.random.seed(0)
    torch.manual_seed(0)
    # memory = get_gpu_memory()

    if cur_dataset is 'OnlineProducts':
        train_classes, test_classes = sop_train_classes, sop_test_classes
        ds_test = OnlineProductsDataset(ds_path=sop_dataset_path, mode='test')
        dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size)
        nb_train_data = sop_nb_train_data

    elif cur_dataset is 'InShopClothes':
        train_classes, test_classes = inshop_train_classes, inshop_test_classes
        ds_query = InShopDataset(ds_path=inshop_dataset_path, mode='query')
        ds_gallery = InShopDataset(ds_path=inshop_dataset_path, mode='gallery')
        dl_query = torch.utils.data.DataLoader(ds_query, batch_size=batch_size)
        dl_gallery = torch.utils.data.DataLoader(ds_gallery, batch_size=batch_size)
        nb_train_data = inshop_nb_train_data

    elif cur_dataset is 'CUB200':
        train_classes, test_classes = cub_train_classes, cub_test_classes
        ds_test = CUBDataset(ds_path=cub_dataset_path, mode='test')
        dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size)
        nb_train_data = cub_nb_train_data

    elif cur_dataset is 'CARS196':
        train_classes, test_classes = cars_train_classes, cars_test_classes
        ds_test = CARSDataset(ds_path=cars_dataset_path, mode='test')
        dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size)
        nb_train_data = cars_nb_train_data

    model_ = get_model().cuda()
    criterion_ = get_criterion()
    all_betas = [criterion_[i].beta for i in range(len(criterion_))]
    optimizer_ = get_optimizer(model_, all_betas)
#     model_, optimizer_ = amp.initialize(model_, optimizer_, opt_level='O1')

    #     test(model_, 186)

    total_training_start_time = time.time()
    train(model_, criterion_, optimizer_, all_betas)
    print(f'Training time: {round((time.time() - total_training_start_time) / 60, 3)} min')

    # test(model_, 0)